# Training on GPU cluster
One of the advantages of running a cluster, is that you can have relatively small instance running for the notebook server and then send of the training task to a separate compute cluster, which can be running powerful (and expensive) GPUs, which spin up to run the training script and then spin down.

In [1]:
%matplotlib inline

In [2]:
import pandas as pd
import numpy as np
import pathlib
import matplotlib.pyplot as plt

In [3]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

In [5]:
import prd_pipeline

In [6]:
import importlib

In [41]:
importlib.reload(prd_pipeline)

<module 'prd_pipeline' from '/mnt/batch/tasks/shared/LS_root/mounts/clusters/prd-ml-pipeline/code/Users/stephen.haddad/precip_rediagnosis/model_pipeline/prd_pipeline.py'>

## set up azure experiment


In [8]:
import azureml.core
from azureml.core import Workspace, Datastore, Dataset, Environment
from azureml.core import Experiment, ComputeTarget, ScriptRunConfig

In [9]:
import fsspec

In [10]:
import pandas

In [11]:
import json

In [12]:
with open('credentials_file.json') as credentials_file:
    az_blob_cred = json.load(credentials_file)
    
az_blob_cred.keys()

dict_keys(['storage_acc_name', 'storage_acc_key', 'container'])

In [13]:
container = az_blob_cred['container']
acc_name = az_blob_cred['storage_acc_name']
acc_key = az_blob_cred['storage_acc_key']


In [39]:
# prd_rel_path = 'prd/202008_storm_francis/prd_merged_20200824T1800Z_20200826T1200Z.csv'
prd_rel_path = 'prd/202008_storm_francis/prd_merged*csv'

In [15]:
prd_data_url = f'abfs://{container}/{prd_rel_path}'
prd_data_url

'abfs://training202209/prd/202008_storm_francis/prd_merged_20200824T1800Z_20200826T1200Z.csv'

In [16]:
%%time
fsspec_handle = fsspec.open(
    prd_data_url,
    account_name=acc_name, 
    account_key=acc_key
)
with fsspec_handle.open() as prd_data_handle:
    prd_merged_data = pandas.read_csv(prd_data_handle)

In [42]:
prd_pipeline.load_data_azure_blob(az_blob_cred, prd_rel_path)

,time,latitude,longitude,radar_max_rain_aggregate_3hr,radar_mean_rain_aggregate_3hr,radar_max_rain_instant,radar_mean_rain_instant,fraction_sum_agg,fraction_sum_instant,radar_fraction_in_band_aggregate_3hr_0.0,...,wind_speed_2750.0,wind_speed_3000.0,wind_speed_3250.0,wind_speed_3500.0,wind_speed_3750.0,wind_speed_4000.0,wind_speed_4500.0,wind_speed_5000.0,wind_speed_5500.0,wind_speed_6000.0
0,2020-08-24 18:00:00,49.40625,-5.484375,7.705729,4.288671,3.4375,0.362647,1.0,1.042353,0.0,...,15.2500,16.2500,16.6875,16.5625,16.2500,16.0625,16.7500,18.3125,20.8750,24.1875
1,2020-08-24 18:00:00,49.40625,-5.484375,7.705729,4.288671,3.4375,0.362647,1.0,1.042353,0.0,...,12.9375,12.5000,12.0625,11.7500,11.7500,12.1875,14.6250,18.1875,20.8125,23.5625
2,2020-08-24 18:00:00,49.40625,-5.484375,7.705729,4.288671,3.4375,0.362647,1.0,1.042353,0.0,...,15.2500,16.3125,17.6250,18.7500,19.3125,19.2500,18.6875,19.6250,22.1875,25.1875
3,2020-08-24 18:00:00,49.40625,-5.484375,7.705729,4.288671,3.4375,0.362647,1.0,1.042353,0.0,...,11.4375,14.3750,14.3125,13.0625,12.3750,13.0625,14.0625,16.0625,19.0625,22.1250
4,2020-08-24 18:00:00,49.40625,-5.484375,7.705729,4.288671,3.4375,0.362647,1.0,1.042353,0.0,...,16.0625,16.2500,16.3125,16.5625,16.9375,17.5000,18.6875,20.7500,23.7500,26.2500
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
213691,2020-08-26 12:00:00,58.78125,1.546875,0.000000,0.000000,0.0000,0.000000,1.0,1.000000,1.0,...,6.1250,5.5625,5.1875,5.1250,5.3125,5.5625,6.0625,8.2500,8.8750,8.3125
213692,2020-08-26 12:00:00,58.78125,1.546875,0.000000,0.000000,0.0000,0.000000,1.0,1.000000,1.0,...,6.5625,6.8125,6.6875,6.0000,5.0000,3.9375,3.2500,5.0000,5.1875,5.5625
213693,2020-08-26 12:00:00,58.78125,1.546875,0.000000,0.000000,0.0000,0.000000,1.0,1.000000,1.0,...,4.4375,3.6875,3.1250,3.0000,3.4375,4.1250,5.5000,8.2500,9.1250,9.5000
213694,2020-08-26 12:00:00,58.78125,1.546875,0.000000,0.000000,0.0000,0.000000,1.0,1.000000,1.0,...,2.9375,3.7500,4.0000,3.5000,2.7500,2.3750,3.0625,4.5000,5.3750,8.8750


In [33]:
%%time
handle1 = fsspec.open_files(
    'abfs://training202209/prd/*/prd_merged*csv',
    account_name=acc_name, 
    account_key=acc_key
)


CPU times: user 104 ms, sys: 2.58 ms, total: 107 ms
Wall time: 326 ms


In [27]:
with list(handle1)[0].open() as prd_dh:
    df1 = pandas.read_csv(prd_dh)
df1

,realization,latitude,longitude,forecast_period,forecast_reference_time,time,cloud_area_fraction,surface_altitude,air_pressure_at_sea_level,rainfall_rate,...,radar_fraction_in_band_aggregate_3hr_0.0,radar_fraction_in_band_aggregate_3hr_0.25,radar_fraction_in_band_aggregate_3hr_2.5,radar_fraction_in_band_aggregate_3hr_7.0,radar_fraction_in_band_aggregate_3hr_10.0,radar_fraction_in_band_instant_0.0,radar_fraction_in_band_instant_0.25,radar_fraction_in_band_instant_2.5,radar_fraction_in_band_instant_7.0,radar_fraction_in_band_instant_10.0
0,0,49.40625,-5.484375,0 days 06:00:00,2020-02-07 12:00:00,2020-02-07 18:00:00,0.984375,0.0,101050.0,0.000000,...,0.0,0.807059,0.197647,0.0,0.0,0.463529,0.515294,0.025882,0.0,0.0
1,1,49.40625,-5.484375,0 days 06:00:00,2020-02-07 12:00:00,2020-02-07 18:00:00,1.000000,0.0,101036.0,0.536442,...,0.0,0.807059,0.197647,0.0,0.0,0.463529,0.515294,0.025882,0.0,0.0
2,2,49.40625,-5.484375,0 days 06:00:00,2020-02-07 12:00:00,2020-02-07 18:00:00,0.984375,0.0,101104.0,0.000000,...,0.0,0.807059,0.197647,0.0,0.0,0.463529,0.515294,0.025882,0.0,0.0
3,3,49.40625,-5.484375,0 days 06:00:00,2020-02-07 12:00:00,2020-02-07 18:00:00,0.843750,0.0,101005.0,0.000000,...,0.0,0.807059,0.197647,0.0,0.0,0.463529,0.515294,0.025882,0.0,0.0
4,4,49.40625,-5.484375,0 days 06:00:00,2020-02-07 12:00:00,2020-02-07 18:00:00,1.000000,0.0,101038.0,0.000000,...,0.0,0.807059,0.197647,0.0,0.0,0.463529,0.515294,0.025882,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
347251,13,58.78125,1.546875,0 days 06:00:00,2020-02-10 12:00:00,2020-02-10 18:00:00,1.000000,0.0,96907.0,0.000000,...,1.0,0.000000,0.000000,0.0,0.0,1.000000,0.000000,0.000000,0.0,0.0
347252,14,58.78125,1.546875,0 days 06:00:00,2020-02-10 12:00:00,2020-02-10 18:00:00,0.812500,0.0,96895.0,0.000000,...,1.0,0.000000,0.000000,0.0,0.0,1.000000,0.000000,0.000000,0.0,0.0
347253,15,58.78125,1.546875,0 days 06:00:00,2020-02-10 12:00:00,2020-02-10 18:00:00,0.984375,0.0,96989.0,0.000000,...,1.0,0.000000,0.000000,0.0,0.0,1.000000,0.000000,0.000000,0.0,0.0
347254,16,58.78125,1.546875,0 days 06:00:00,2020-02-10 12:00:00,2020-02-10 18:00:00,1.000000,0.0,96823.0,0.000000,...,1.0,0.000000,0.000000,0.0,0.0,1.000000,0.000000,0.000000,0.0,0.0


In [34]:
%%time
csv_list = []
for h1 in list(handle1)[:3]:
    with h1.open() as prd_dh:
        csv_list += [pandas.read_csv(prd_dh)]

df1 = pandas.concat(csv_list)

<timed exec>:6: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.



CPU times: user 42.6 s, sys: 6.89 s, total: 49.5 s
Wall time: 1min 21s


In [35]:
len(csv_list)

3

In [36]:
[df2.shape for df2 in csv_list]

[(347256, 227), (347256, 227), (373968, 227)]

In [37]:
df1.shape

(1068480, 227)

In [ ]:
fsspec.__version__

In [ ]:
import adlfs
adlfs.__version__

In [ ]:
azureml.core.Dataset.get_by_name(prd_ws, 'prd_merged_202112_storm_barra_files').as_download()

In [ ]:
fsspec_handle

In [ ]:
prd_merged_data

In [ ]:
prd_ws = Workspace.from_config()

In [ ]:
load_all = False
if load_all:
    azure_dataset_name ='prd_merged_all_events_files'
else:
    #  use subset for development.
    azure_dataset_name ='prd_merged_202110_nswws_amber_oct_files'

In [ ]:
azure_experiment_name='prd_mlops_test'
azure_env_name = 'prd_ml_cluster'
cluster_name = 'mlops-gpu-test'

In [ ]:
prd_model_name = 'azml_cluster_demo_20220414'

In [ ]:
merged_prefix = 'prd_merged'
csv_file_suffix = 'csv'

In [ ]:
target_parameter = 'radar_mean_rain_instant'
profile_features = ['air_temperature', 'relative_humidity']
single_lvl_features = ['air_pressure_at_sea_level'] 

In [ ]:
prd_dataset_all = Dataset.get_by_name(prd_ws, azure_dataset_name)
prd_dataset_all

In [ ]:
import pandas

In [ ]:
with azml_ds.mount() as prd_ds_mount:
    prd_path_list = [p1 for p1 in pathlib.Path(prd_ds_mount.mount_point).rglob(f'{merged_prefix}*{csv_file_suffix}') ]
    merged_df = pandas.concat([pandas.read_csv(p1) for p1 in prd_path_list])

In [ ]:
merged_df

In [ ]:
[c1 for c1 in merged_df.columns if 'rain' in c1]

In [ ]:
prd_exp = Experiment(workspace=prd_ws, name=azure_experiment_name)
prd_exp

Get the AzML environment (basically a conda environment) from the workspace.

In [ ]:
prd_env = Environment.get(workspace=prd_ws, name=azure_env_name)
prd_env

### Load data

load the data from the script so we'renot duplicating code

In [ ]:
# import importlib 
# importlib.reload(prd_cluster_train_demo)

In [ ]:
%%time
input_data = prd_pipeline.load_data(
    prd_ws,
    dataset_name=azure_dataset_name
)
data_splits, data_dims = prd_pipeline.preprocess_data(
    input_data,
    feature_dict={'profile': profile_features, 'single_level': single_lvl_features,'target': target_parameter,},
)


In [ ]:
# these are example calls to the code for easier debugging than running on a separate cluster
# model = prd_cluster_train_demo.build_model(**data_dims)
# model = prd_cluster_train_demo.train_model(model, data_splits)

In [ ]:
import datetime
log_dir = 'log/fit/' + datetime.datetime.now().strftime('%Y%m%d-%H%M%S')

### Execute our training run on a cluster

In [ ]:
prd_demo_compute_target = ComputeTarget(workspace=prd_ws, name=cluster_name)
prd_demo_compute_target

In [ ]:
prd_demo_args = ['--dataset-name', azure_dataset_name,
                 '--target-parameter', target_parameter,
                 '--model-name', prd_model_name,
                ]

prd_demo_args += ['--profile-features']
prd_demo_args += profile_features
prd_demo_args += ['--single-level_features']
prd_demo_args += single_lvl_features

prd_demo_args

In [ ]:
prd_run_src = ScriptRunConfig(source_directory=os.getcwd(),
                      script='prd_cluster_train_demo.py',
                      arguments=prd_demo_args,
                      compute_target=prd_demo_compute_target,
                      environment=prd_env)

In [ ]:
prd_run = prd_exp.submit(prd_run_src)
prd_run

## Evaluation

We now get the trained model to do some evaluatiion and create some plots

In [ ]:
import tempfile

In [ ]:
import tensorflow.keras

We download the model file into a temproary directory (so as not to pollute the local workspace) and load into memory to do inference

In [ ]:
with tempfile.TemporaryDirectory() as td1:
    td_path = pathlib.Path(td1)
    prd_run.download_files(prefix=prd_model_name, output_directory=td1)
    model_path = td_path / prd_model_name
    list(model_path.iterdir())
    trained_model = tensorflow.keras.models.load_model(model_path)

In [ ]:
trained_model

In [ ]:
data_splits.keys()

In [ ]:
y_pred = trained_model.predict(data_splits['X_test'])

In [ ]:
fig1 = plt.figure(figsize=(10, 8))
ax1 = fig1.add_subplot(1,1,1)
ax1.scatter(data_splits['y_test'], y_pred, s=200, c='darkblue')
ax1.plot([0, 300], [0, 300], ls="--", c=".3")
ax1.set_xlabel('Actual 3hr precip accumulation value')
ax1.set_ylabel('Predicted 3hr precip_accumulation value')

In [ ]:
prd_run.log_image(name='actual_vs_pred', plot=fig1, description='predicted vs actual 3hr accumulations of rainfall')

In [ ]:
prd_run.complete()